In [19]:
# import modules
import numpy as np
import pandas as pd

from scipy import stats
from scipy.stats import norm, skew
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
#from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor

#import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

In [18]:
# load dataset
train = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")
train.shape

(1460, 81)

In [3]:
# extract labels and index
length = len(train)
train_id = train["Id"]
test_id = test["Id"]
train_label = train["SalePrice"]

full = pd.concat([train, test], axis = 0).reset_index().drop(columns=["Id", "index", "SalePrice"])

In [4]:
# observe ategorical data distribustion
cat_cols = [col for col in full.select_dtypes(include="object").columns]
def cols():
    for col in cat_cols:
        yield col, full[col].value_counts(dropna=False)
cols = cols()

for col in cols:
    print(col)

('Alley', NaN     2721
Grvl     120
Pave      78
Name: Alley, dtype: int64)
('BldgType', 1Fam      2425
TwnhsE     227
Duplex     109
Twnhs       96
2fmCon      62
Name: BldgType, dtype: int64)
('BsmtCond', TA     2606
Gd      122
Fa      104
NaN      82
Po        5
Name: BsmtCond, dtype: int64)
('BsmtExposure', No     1904
Av      418
Gd      276
Mn      239
NaN      82
Name: BsmtExposure, dtype: int64)
('BsmtFinType1', Unf    851
GLQ    849
ALQ    429
Rec    288
BLQ    269
LwQ    154
NaN     79
Name: BsmtFinType1, dtype: int64)
('BsmtFinType2', Unf    2493
Rec     105
LwQ      87
NaN      80
BLQ      68
ALQ      52
GLQ      34
Name: BsmtFinType2, dtype: int64)
('BsmtQual', TA     1283
Gd     1209
Ex      258
Fa       88
NaN      81
Name: BsmtQual, dtype: int64)
('CentralAir', Y    2723
N     196
Name: CentralAir, dtype: int64)
('Condition1', Norm      2511
Feedr      164
Artery      92
RRAn        50
PosN        39
RRAe        28
PosA        20
RRNn         9
RRNe         6
Name: Con

In [5]:
# feature processing
def transform(x):
    x = x.replace({"Alley":{np.nan:0, "Grvl":1, "Pave":2},
                   "BsmtCond":{"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,np.nan:0},
                   "BsmtExposure":{"Gd":4,"Av":3,"Mn":2,"No":1,np.nan:0},
                   "BsmtFinType1":{"GLQ":6,"ALQ":5,"BLQ":4,"Rec":3,"LwQ":2,"Unf":1, np.nan:0},
                   "BsmtFinType2":{"GLQ":6,"ALQ":5,"BLQ":4,"Rec":3,"LwQ":2,"Unf":1, np.nan:0},
                   "BsmtQual":{"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,np.nan:0},
                   "CentralAir":{"Y":1,"N":0},
                   "Electrical":{"SBrkr":4,"FuseA":3,"FuseF":2,"FuseP":1,"Mix":0,np.nan:0},
                   "ExterCond":{"Ex":4,"Gd":3,"TA":2,"Fa":1,"Po":0},
                   "Exterior1st":{np.nan:x.Exterior1st.mode()[0]},
                   "Exterior2nd":{np.nan:x.Exterior2nd.mode()[0]},
                   "ExterQual":{"Ex":4,"Gd":3,"TA":2,"Fa":1,"Po":0},
                   "Fence":{np.nan:"None"},
                   "FireplaceQu":{"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,np.nan:0},
                   "Functional":{"Typ":6,"Min1":5,"Min2":5,"Mod":4,"Maj1":3,"Maj2":3,"Sev":2,"Sal":1,np.nan:0},
                   "GarageCond":{"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,np.nan:0},
                   "GarageFinish":{"Fin":3,"RFn":2,"Unf":1,np.nan:0},
                   "GarageQual":{"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,np.nan:0},
                   "GarageType":{np.nan:"None"},
                   "HeatingQC":{"Ex":4,"Gd":3,"TA":2,"Fa":1,"Po":0},
                   "KitchenQual":{"Ex":5,"Gd":4,"TA":3,"Fa":2,"Po":1,np.nan:0},
                   "MSZoning":{np.nan:x.MSZoning.mode()[0]},
                   "MasVnrType":{np.nan:x.MasVnrType.mode()[0]},
                   "MiscFeature":{np.nan:"None"},
                   "PavedDrive":{"Y":2,"P":1,"N":0},
                   "PoolQC":{"Ex":4,"Gd":3,"TA":2,"Fa":1,np.nan:0},
                   "Street":{"Grvl":0, 'Pave':1},
                   "Utilities":{"AllPub":4,"NoSewr":3,"NoSeWa":2,"ELO":1,np.nan:0},
                   "SaleType":{np.nan:x.SaleType.mode()[0]}
                  })
    cat = ["MSSubClass", "BldgType", "Condition1", "Condition2", "Exterior1st", 
           "Exterior2nd", "Fence", "Foundation", "GarageType", "Heating", 
           "HouseStyle", "LandContour", "LandSlope", "LotConfig", "LotShape", 
           "MSZoning", "MasVnrType", "MiscFeature", "Neighborhood", "RoofMatl", 
           "RoofStyle", "SaleCondition", "SaleType", "YearBuilt", "YearRemodAdd",
           "MoSold", "YrSold"]
    x[cat] = x[cat].astype("object")
    num = [col for col in x.columns if col not in cat]
    x[num] = x[num].fillna(0)
    x[num].astype("int", inplace=True)
    
    #ohc = OneHotEncoder(sparse=False)
    #cat_ready = pd.DataFrame(ohc.fit_transform(x[cat]))
    dummies = pd.get_dummies(x, drop_first=True)
    
    mms = MinMaxScaler()
    ready = pd.DataFrame(mms.fit_transform(dummies))
    
    #data_ready = pd.concat([cat_ready, num_ready], axis=1)
    return ready

full = transform(full)
#test = transform(test)

In [23]:
# prepare dataset for ML 
train_pro = full[:length]
test = full[length:]

X_train, y_train, X_test, y_test = train_test_split(train_pro, train_label, test_size=0.2, random_state=1988)
X_train.shape

(1168, 406)

In [7]:
# RMSE for evaluation
def rmse(true, test):
    rmse = np.sqrt(mean_squared_error(true, test))
    return rmse

In [8]:
# classic linear regression
lr = LinearRegression()
lr.fit(X_train, X_test)
pred_lr = lr.predict(y_train)
print(rmse(y_test, pred_lr))

5433138070716057.0


In [9]:
param_grid = {"alpha":np.linspace(100, 10000, 10)}
# lasso
ls = Lasso()
grid_ls = GridSearchCV(ls, param_grid=param_grid, cv=10)
grid_ls.fit(X_train, X_test)
pred_ls = grid_ls.predict(y_train)
print(grid_ls.best_params_)
print(grid_ls.best_score_)
print(rmse(y_test, pred_ls))

{'alpha': 100.0}
0.7685344776388543
24329.956783261598


In [10]:
# ridge
rg = Ridge()
grid_rg = GridSearchCV(rg, param_grid=param_grid, cv=10)
grid_rg.fit(X_train, X_test)
pred_rg = grid_rg.predict(y_train)
print(grid_rg.best_params_)
print(grid_rg.best_score_)
print(rmse(y_test, pred_rg))

{'alpha': 100.0}
0.7306199542527712
37740.45774073547


In [11]:
# tuning XGBoost round 1
# https://www.jianshu.com/p/9abdc030307d
param1 = {'max_depth': list(range(3,10)), 'min_child_weight': list((1,2,3,4,5,6))}
xgb1 = XGBRegressor(eta=0.1, num_boost_round=50, colsample_bytree=0.5, subsample=0.5, seed=1988)
grid_xgb1 = GridSearchCV(xgb1, param_grid=param1, cv=10)
grid_xgb1.fit(X_train, X_test)
pred_xgb1 = grid_xgb1.predict(y_train)
print(grid_xgb1.best_params_)
print(rmse(y_test, pred_xgb1))

[18:56:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:52] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:56:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[18:57:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:57:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[18:58:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:58:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:58:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:58:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:58:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:58:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:58:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:58:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:58:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[18:59:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:59:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:00:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:01:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:02:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:02:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:02:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:02:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:02:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:02:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:02:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:02:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:02:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [12]:
# tuning XGBoost round 2
param2 = {'gamma':np.linspace(0, 0.5, 10)}
xgb2 = XGBRegressor(eta=0.1, num_boost_round=50, colsample_bytree=0.5, subsample=0.5, max_depth=6, min_child_weight=3)
grid_xgb2 = GridSearchCV(xgb2, param_grid=param2, cv=10)
grid_xgb2.fit(X_train, X_test)
pred_xgb2 = grid_xgb2.predict(y_train)
print(grid_xgb2.best_params_)
print(rmse(y_test, pred_xgb2))

[19:04:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:04:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:06:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [13]:
# tuning XGBoost round 3
param3 = {'eta':[0.5,0.4,0.3,0.2,0.1,0.075,0.05,0.04,0.03], "num_boost_round":np.linspace(0, 200, 20)}
xgb3 = XGBRegressor(colsample_bytree=0.5, subsample=0.5, max_depth=6, 
                    min_child_weight=3, gamma=0)
grid_xgb3 = GridSearchCV(xgb3, param_grid=param3, cv=10)
grid_xgb3.fit(X_train, X_test)
pred_xgb3 = grid_xgb3.predict(y_train)
print(grid_xgb3.best_params_)
print(rmse(y_test, pred_xgb3))
# no improvement, XGBoost done

[19:06:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:06:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:07:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:07:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:07:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:07:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:09:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:26] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:09:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:13:00] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:13:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:13:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:13:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:13:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:13:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:13:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:13:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:13:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:14:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:14:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:14:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:14:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:15:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:15:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:15:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:15:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:15:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:16:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:16:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:17:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

...

[19:46:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:46:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:47:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:47:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:49:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:21] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:49:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:50:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:50:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

[19:52:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:30] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:52:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [14]:
# random forest
param_rf = {"n_estimators":range(1, 10), "max_depth":range(1, 10)}
rf = RandomForestRegressor()
grid_rf = GridSearchCV(rf, param_grid=param_rf, cv=10)
grid_rf.fit(X_train, X_test)
pred_rf = grid_rf.predict(y_train)
print(grid_rf.best_params_)
print(rmse(y_test, pred_rf))

{'max_depth': 8, 'n_estimators': 7}
29504.23251119704


In [44]:
# tuning neural network round 1
nn1 = Sequential()
nn1.add(Dense(500, activation="linear", input_shape=(406,)))
nn1.add(Dense(500))
nn1.add(Dense(1))
opt = Adam(lr=1e-3, decay=1e-3 / 200)
nn1.compile(optimizer=opt, loss="mean_squared_error")
nn1.fit(X_train, X_test, epochs=200, batch_size=8)
pred_nn1 = nn1.predict(y_train)
print(rmse(y_test, pred_nn))

Epoch 1/200
1168/1168 [==============================] - 1s 493us/sample - loss: 29812867779.5068
Epoch 2/200
1168/1168 [==============================] - 0s 370us/sample - loss: 4646967218.1918
Epoch 3/200
1168/1168 [==============================] - 0s 378us/sample - loss: 3547292728.3288
Epoch 4/200
1168/1168 [==============================] - 0s 425us/sample - loss: 2927686016.2192
Epoch 5/200
1168/1168 [==============================] - 1s 469us/sample - loss: 2490215877.2603
Epoch 6/200
1168/1168 [==============================] - 1s 518us/sample - loss: 2168074799.2329
Epoch 7/200
1168/1168 [==============================] - 1s 597us/sample - loss: 1939634212.0548
Epoch 8/200
1168/1168 [==============================] - 1s 716us/sample - loss: 1761775965.1507
Epoch 9/200
1168/1168 [==============================] - 1s 936us/sample - loss: 1652209253.3151
Epoch 10/200
1168/1168 [==============================] - 3s 2ms/sample - loss: 1549396589.3699
Epoch 11/200
1168/1168 [======

1168/1168 [==============================] - 2s 2ms/sample - loss: 729342248.4658
Epoch 87/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 726322312.8219
Epoch 88/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 747425187.5616
Epoch 89/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 741449127.1781
Epoch 90/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 719188159.3699
Epoch 91/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 728600164.6301
Epoch 92/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 724544808.1644
Epoch 93/200
1168/1168 [==============================] - 3s 2ms/sample - loss: 724430398.3562
Epoch 94/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 719503839.3973
Epoch 95/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 729995129.3973
Epoch 96/200
1168/1168 [==============================] - 2s 2m

1168/1168 [==============================] - 2s 2ms/sample - loss: 668943509.9178
Epoch 171/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 670326496.9589
Epoch 172/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 673296494.7123
Epoch 173/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 663214155.5616
Epoch 174/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 665483028.8767
Epoch 175/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 665850091.1781
Epoch 176/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 667501938.5205
Epoch 177/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 665787365.7534
Epoch 178/200
1168/1168 [==============================] - 3s 2ms/sample - loss: 670039319.7808
Epoch 179/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 662462264.6301
Epoch 180/200
1168/1168 [=============================

In [42]:
# tuning neural network round 2
nn2 = Sequential()
nn2.add(Dense(1000, activation="linear", input_shape=(406,)))
nn2.add(Dense(1000))
nn2.add(Dense(1))
opt = Adam(lr=1e-3, decay=1e-3 / 200)
nn2.compile(optimizer=opt, loss="mean_squared_error")
nn2.fit(X_train, X_test, epochs=200, batch_size=8)
pred_nn2 = nn2.predict(y_train)
print(rmse(y_test, pred_nn))
# no improvement, done

Epoch 1/200
1168/1168 [==============================] - 1s 1ms/sample - loss: 21325217590.3562
Epoch 2/200
1168/1168 [==============================] - 1s 876us/sample - loss: 3696764349.1507
Epoch 3/200
1168/1168 [==============================] - 1s 866us/sample - loss: 2767004187.5068
Epoch 4/200
1168/1168 [==============================] - 1s 870us/sample - loss: 2199658593.5342
Epoch 5/200
1168/1168 [==============================] - 1s 894us/sample - loss: 1881814989.1507
Epoch 6/200
1168/1168 [==============================] - 1s 1ms/sample - loss: 1661512802.0274
Epoch 7/200
1168/1168 [==============================] - 1s 1ms/sample - loss: 1507732077.7534
Epoch 8/200
1168/1168 [==============================] - 1s 1ms/sample - loss: 1412682459.2877
Epoch 9/200
1168/1168 [==============================] - 2s 1ms/sample - loss: 1328570558.5753
Epoch 10/200
1168/1168 [==============================] - 2s 2ms/sample - loss: 1264897658.6301
Epoch 11/200
1168/1168 [================

1168/1168 [==============================] - 5s 4ms/sample - loss: 717636915.9178
Epoch 85/200
1168/1168 [==============================] - 5s 5ms/sample - loss: 722924274.9589
Epoch 86/200
1168/1168 [==============================] - 5s 4ms/sample - loss: 705753233.3699
Epoch 87/200
1168/1168 [==============================] - 6s 6ms/sample - loss: 709095063.8356 0s - loss: 708862438.124
Epoch 88/200
1168/1168 [==============================] - 7s 6ms/sample - loss: 715525552.3014 1
Epoch 89/200
1168/1168 [==============================] - 5s 5ms/sample - loss: 698148626.9041
Epoch 90/200
1168/1168 [==============================] - 5s 4ms/sample - loss: 706689656.4658
Epoch 91/200
1168/1168 [==============================] - 5s 4ms/sample - loss: 701364334.3836
Epoch 92/200
1168/1168 [==============================] - 5s 4ms/sample - loss: 691363985.1781
Epoch 93/200
1168/1168 [==============================] - ETA: 0s - loss: 710371140.965 - 5s 4ms/sample - loss: 708243412.4932
Epoc

1168/1168 [==============================] - 5s 4ms/sample - loss: 658358807.9452
Epoch 166/200
1168/1168 [==============================] - 5s 4ms/sample - loss: 659479156.0548
Epoch 167/200
1168/1168 [==============================] - 4s 4ms/sample - loss: 691335689.1507
Epoch 168/200
1168/1168 [==============================] - 4s 4ms/sample - loss: 662564093.9452
Epoch 169/200
1168/1168 [==============================] - 5s 4ms/sample - loss: 688119889.8082 0s - loss: 692060500.944
Epoch 170/200
1168/1168 [==============================] - 5s 4ms/sample - loss: 653440577.7534
Epoch 171/200
1168/1168 [==============================] - 4s 4ms/sample - loss: 654347196.2192
Epoch 172/200
1168/1168 [==============================] - 4s 4ms/sample - loss: 654924866.6575
Epoch 173/200
1168/1168 [==============================] - 4s 4ms/sample - loss: 673992640.6027
Epoch 174/200
1168/1168 [==============================] - 5s 4ms/sample - loss: 659081078.3014
Epoch 175/200
1168/1168 [====

In [49]:
pred = grid_ls.predict(test) * 0.7 + grid_xgb3.predict(test) * 0.3 #+ grid_rf.predict(test) * 0.1 +nn1.predict(test).T * 0.1
# random forest and neural network don't improve

In [50]:
score = pd.DataFrame()
score["Id"] = test_id
score["SalePrice"] = pred.T
score.to_csv('submission.csv',index=False)